# Actividad 1: HDFS, Spark SQL y MLlib

## Recuerda borrar siempre las líneas que dicen `raise NotImplementedError`

In [ ]:
Alumno: Leonard Jose Cuenca Roa
Carrera: Analisis y visualización de Datos Big Data 
Fecha: 13/07/2025

Lee con detenimiento cada ejercicio. Las variables utilizadas para almacenar las soluciones, al igual que las nuevas columnas creadas, deben llamarse **exactamente** como indica el ejercicio, o de lo contrario los tests fallarán y el ejercicio no puntuará. Debe reemplazarse el valor `None` al que están inicializadas por el código necesario para resolver el ejercicio.

## Leemos el fichero flights.csv que hemos subido a HDFS

Indicamos que contiene encabezados (nombres de columnas) y que intente inferir el esquema, aunque después comprobaremos si lo
ha inferido correctamente o no. La ruta del archivo en HDFS debería ser /<nombre_alumno>/flights.csv

In [55]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import IntegerType, DoubleType

spark = SparkSession.builder.appName("MiPrimeraAppSpark").getOrCreate()

ruta_hdfs = "/CuencaRoaLeonardJose/flights.csv"
# Descomentar estas líneas
flightsDF = spark.read.csv(ruta_hdfs, header="true", inferSchema="true")

# 1. Imprimo el esquema del DataFrame para revisar los tipos de datos
print("Esquema del DataFrame:")
flightsDF.printSchema()

# 2. Muestro las primeras 5 filas para inspeccionar los datos
print("\nPrimeras 5 filas del DataFrame:")
flightsDF.show(5)

# 3. Mostramos el número de filas que tiene el DataFrame para hacernos una idea de su tamaño:
flightsDF.count()

# 4. Vamos a averiguar cuántas filas tienen el valor "NA" (como string) en la columna dep_time:
cuantos_NA = flightsDF\
                .where(F.col("dep_time") == "NA")\
                .count()
cuantos_NA

# 5. En nuestro caso, como tenemos un número considerable de filas, vamos a quitar todas las filas donde hay un NA en cualquiera de las columnas.
columnas_limpiar = ["dep_time", "dep_delay", "arr_time", "arr_delay", "air_time", "hour", "minute"]

flightsLimpiado = flightsDF
for nombreColumna in columnas_limpiar:  # para cada columna, nos quedamos con las filas que no tienen NA en esa columna
    flightsLimpiado = flightsLimpiado.where(F.col(nombreColumna) != "NA")

flightsLimpiado.cache()

# 6. Si ahora mostramos el número de filas que tiene el DataFrame flightsLimpiado tras eliminar todas esas filas, vemos que ha disminuido ligeramente pero sigue siendo un número considerable como para realizar analítica y sacar conclusiones sobre estos datos
flightsLimpiado.count()

# 7.  Una vez que hemos eliminado los NA, vamos a convertir a tipo entero cada una de esas columnas que eran de tipo string.

flightsConvertido = flightsLimpiado

for c in columnas_limpiar:
    # método que crea una columna o reemplaza una existente
    flightsConvertido = flightsConvertido.withColumn(c, F.col(c).cast(IntegerType())) 

flightsConvertido = flightsConvertido.withColumn("arr_delay", F.col("arr_delay").cast(DoubleType()))
flightsConvertido.cache()

flightsConvertido.printSchema()

flightsConvertido.show(5)


Esquema del DataFrame:
root
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- dep_time: string (nullable = true)
 |-- dep_delay: string (nullable = true)
 |-- arr_time: string (nullable = true)
 |-- arr_delay: string (nullable = true)
 |-- carrier: string (nullable = true)
 |-- tailnum: string (nullable = true)
 |-- flight: integer (nullable = true)
 |-- origin: string (nullable = true)
 |-- dest: string (nullable = true)
 |-- air_time: string (nullable = true)
 |-- distance: integer (nullable = true)
 |-- hour: string (nullable = true)
 |-- minute: string (nullable = true)


Primeras 5 filas del DataFrame:
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+--

### Ejercicio 1

Partiendo del DataFrame `flightsConvertido` que ya tiene los tipos correctos en las columnas, se pide: 

* Crear un nuevo DataFrame llamado `aeropuertosOrigenDF` que tenga una columna `origin` y que tenga tantas filas como aeropuertos distintos de *origen* existan. ¿Cuántas filas tiene? Almacenar dicho recuento en la variable entera `n_origen`.
* Crear un nuevo DataFrame llamado `rutasDistintasDF` que tenga dos columnas `origin`, `dest` y que tenga tantas filas como rutas diferentes existan (es decir, como combinaciones distintas haya entre un origen y un destino). Una vez creado, contar cuántas combinaciones hay, almacenando dicho recuento en la variable entera `n_rutas`.


In [56]:
# Contar aeropuertos de origen distintos
aeropuertosOrigenDF = flightsConvertido.select("origin").distinct()
n_origen = aeropuertosOrigenDF.count()
print(f"Número de aeropuertos de origen distintos: {n_origen}")
aeropuertosOrigenDF.show()

# Contar rutas distintas (combinaciones de origen y destino)
rutasDistintasDF  = flightsConvertido.select("origin", "dest").distinct()
n_rutas = rutasDistintasDF.count()

print(f"Número de rutas distintas: {n_rutas}")
rutasDistintasDF.show()

Número de aeropuertos de origen distintos: 2
+------+
|origin|
+------+
|   SEA|
|   PDX|
+------+

Número de rutas distintas: 115
+------+----+
|origin|dest|
+------+----+
|   SEA| RNO|
|   SEA| DTW|
|   SEA| CLE|
|   SEA| LAX|
|   PDX| SEA|
|   SEA| BLI|
|   PDX| IAH|
|   PDX| PHX|
|   SEA| SLC|
|   SEA| SBA|
|   SEA| BWI|
|   PDX| IAD|
|   PDX| SFO|
|   SEA| KOA|
|   SEA| JAC|
|   PDX| MCI|
|   SEA| SJC|
|   SEA| ABQ|
|   SEA| SAT|
|   PDX| ONT|
+------+----+
only showing top 20 rows



In [57]:
assert(n_origen == 2)
assert(n_rutas == 115)
assert(aeropuertosOrigenDF.count() == n_origen)
assert(rutasDistintasDF.count() == n_rutas)

### Ejercicio 2

* Partiendo de nuevo de `flightsConvertido`, se pide calcular, *sólo para los vuelos que llegan con* ***retraso positivo***, el retraso medio a la llegada de dichos vuelos, para cada aeropuerto de destino. La nueva columna con el retraso medio a la llegada debe llamarse `retraso_medio`. El DF resultante debe estar **ordenado de mayor a menor retraso medio**. El código que calcule esto debería ir encapsulado en una función de Python llamada `retrasoMedio` que reciba como argumento un DataFrame y devuelva como resultado el DataFrame con el cálculo descrito anteriormente.

* Una vez hecha la función, invocarla pasándole como argumento `flightsConvertido` y almacenar el resultado devuelto en la variable `retrasoMedioDF`.

In [58]:
def retrasoMedio(df):
    # 1. Primero Filtro vuelos con retraso positivo
    vuelos_con_retraso_positivo = df.where(F.col("arr_delay") > 0)

    # 2. Segundo: Agrupo por aeropuerto de destino y 
    # 3. Tercero: Calculo el retraso medio
    # 4. Cuarto: Renombro la columna del retraso medio
    retraso_medio_df = vuelos_con_retraso_positivo.groupBy("dest") \
                                                  .agg(F.avg("arr_delay").alias("retraso_medio"))

    # 5. Quinto: Ordeno el DataFrame resultante de mayor a menor retraso medio
    df_final_ordenado = retraso_medio_df.orderBy(F.col("retraso_medio").desc())

    return df_final_ordenado

In [59]:
lista = retrasoMedio(flightsConvertido).take(3)
assert((lista[0].retraso_medio == 64.75) & (lista[0].dest == "BOI"))
assert((lista[1].retraso_medio == 46.8) & (lista[1].dest == "HDN"))
assert((round(lista[2].retraso_medio, 2) == 41.19) & (lista[2].dest == "SFO"))

Ahora invocamos a nuestra función `retrasoMedio` pasándole como argumento `flightsConvertido`. ¿Cuáles son los tres aeropuertos con mayor retraso medio? ¿Cuáles son sus retrasos medios en minutos?

In [60]:
# Invocar la función con tu DataFrame flightsConvertido
retrasoMedioDF = retrasoMedio(flightsConvertido)

# Mostrar los tres primeros resultados como esta ordenado podremos mostrar los primeros tres de esta manera
print("Los tres aeropuertos con mayor retraso medio y sus retrasos:")
retrasoMedioDF.show(3)


Los tres aeropuertos con mayor retraso medio y sus retrasos:
+----+------------------+
|dest|     retraso_medio|
+----+------------------+
| BOI|             64.75|
| HDN|              46.8|
| SFO|41.193768844221104|
+----+------------------+
only showing top 3 rows



### Ejercicio 3

Ajustar un modelo de DecisionTree de Spark para predecir si un vuelo vendrá o no con retraso (problema de clasificación binaria), utilizando como variables predictoras el mes, el día del mes, la hora de partida `dep_time`, la hora de llegada `arr_time`, el tipo de avión (`carrier`), la distancia y el tiempo que permanece en el aire. Para ello, sigue los siguientes pasos.

Notemos que en estos datos hay variables numéricas y variables categóricas que ahora mismo están tipadas como numéricas, como por ejemplo el mes del año (`month`), que es en realidad categórica. Debemos indicar a Spark cuáles son categóricas e indexarlas. Para ello se pide: 

* Crear un `StringIndexer` llamado `indexerMonth` y otro llamado `indexerCarrier` sobre las variables categóricas `month` y `carrier` (tipo de avión). El nombre de las columnas indexadas que se crearán debe ser, respectivamente, `monthIndexed` y `carrierIndexed`.

In [61]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.sql import functions as F

# 0. Preparo la columna 'label' 
flights_con_label = flightsConvertido.withColumn("label", F.when(F.col("arr_delay") > 0, 1).otherwise(0))

# 1. Defino los StringIndexers
indexerMonth = StringIndexer(inputCol="month", outputCol="monthIndexed")
indexerCarrier = StringIndexer(inputCol="carrier", outputCol="carrierIndexed")

# 2. Defino las columnas a usar en el VectorAssembler (incluyendo las indexadas)
feature_cols = ["monthIndexed", "day", "dep_time", "arr_time", "carrierIndexed", "distance", "air_time"]
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")

# 3. Defino el modelo Decision Tree
dt = DecisionTreeClassifier(labelCol="label", featuresCol="features", seed=42)

# 4. Genero el Pipeline (orden de ejecución: indexers -> assembler -> dt)
pipeline = Pipeline(stages=[indexerMonth, indexerCarrier, assembler, dt])

# 5. Divido los datos (antes de fit_pipeline si el pipeline incluye el modelo)
train_data, test_data = flights_con_label.randomSplit([0.7, 0.3], seed=42)

# 6. Ajusto y entreno el Pipeline completo con los datos de entrenamiento
model = pipeline.fit(train_data)

# 7. Realizo las predicciones en el conjunto de prueba
prediciones = model.transform(test_data)

# 8. Evaluo el modelo
evaluator = BinaryClassificationEvaluator(labelCol="label", rawPredictionCol="rawPrediction", metricName="areaUnderROC")
auc = evaluator.evaluate(prediciones)

# Muestro las prediciones 
print(f"Área bajo la curva ROC (AUC) = {auc}")
prediciones.select("label", "prediction", "probability", "rawPrediction", "arr_delay").show(5)

Área bajo la curva ROC (AUC) = 0.4876462417378798
+-----+----------+--------------------+---------------+---------+
|label|prediction|         probability|  rawPrediction|arr_delay|
+-----+----------+--------------------+---------------+---------+
|    0|       0.0|[0.73400176938956...|[7467.0,2706.0]|     -4.0|
|    1|       0.0|[0.73400176938956...|[7467.0,2706.0]|    219.0|
|    1|       1.0|[0.48260869565217...|  [444.0,476.0]|     24.0|
|    0|       0.0|[0.73400176938956...|[7467.0,2706.0]|     -6.0|
|    0|       0.0|[0.65965814551505...|[8722.0,4500.0]|    -25.0|
+-----+----------+--------------------+---------------+---------+
only showing top 5 rows



In [62]:
assert(isinstance(indexerMonth, StringIndexer))
assert(isinstance(indexerCarrier, StringIndexer))
assert(indexerMonth.getInputCol() == "month")
assert(indexerMonth.getOutputCol() == "monthIndexed")
assert(indexerCarrier.getInputCol() == "carrier")
assert(indexerCarrier.getOutputCol() == "carrierIndexed")

Recordemos también que Spark requiere que todas las variables estén en una única columna de tipo vector, por lo que después de indexar estas dos variables, tendremos que fusionar en una columna de tipo vector todas ellas, utilizando un `VectorAssembler`. Se pide:

* Crear en una variable llamada `vectorAssembler` un `VectorAssembler` que reciba como entrada una lista de todas las variables de entrada (y que no debe incluir `arr_delay`) que serán las que formarán parte del modelo. Crear primero esta lista de variables (lista de strings) en la variable `columnas_ensamblar` y pasar dicha variable como argumento al crear el `VectorAssembler`. Como es lógico, en el caso de las columnas `month` y `carrier`, no usaremos las variables originales sino las indexadas en el apartado anterior. La columna de tipo vector creada con las características ensambladas debe llamarse `features`.

In [63]:
from pyspark.ml.feature import VectorAssembler

# Paso 1: Crear la lista de variables de entrada (strings)
# Asegúrate de usar las columnas indexadas para 'month' y 'carrier'
columnas_ensamblar = [
    "monthIndexed",   # Columna indexada
    "day",
    "dep_time",
    "arr_time",
    "carrierIndexed", # Columna indexada
    "distance",
    "air_time"
]

# Paso 2: Crear el VectorAssembler
# La columna de salida se llamará "features"
vectorAssembler = VectorAssembler(inputCols=columnas_ensamblar, outputCol="features")

print(vectorAssembler.getOutputCol())
print(vectorAssembler.getInputCols())

features
['monthIndexed', 'day', 'dep_time', 'arr_time', 'carrierIndexed', 'distance', 'air_time']


In [64]:
assert(isinstance(vectorAssembler, VectorAssembler))
assert(vectorAssembler.getOutputCol() == "features")
input_cols = vectorAssembler.getInputCols()
assert(len(input_cols) == 7)
assert("arr_delay" not in input_cols)

Finalmente, vemos que la columna `arr_delay` es continua, y no binaria como requiere un problema de clasificación con dos clases. Vamos a convertirla en binaria. Para ello se pide:

* Utilizar un binarizador de Spark, fijando a 15 el umbral, y guardarlo en la variable `delayBinarizer`. Consideramos retrasado un vuelo que ha llegado con más de 15 minutos de retraso, y no retrasado en caso contrario. La nueva columna creada con la variable binaria debe llamarse `arr_delay_binary` y debe ser interpretada como la columna target para nuestro algoritmo. Por ese motivo, esta columna **no** se incluyó en el apartado anterior entre las columnas que se ensamblan para formar las features.

In [65]:
from pyspark.ml.feature import Binarizer
from pyspark.sql import functions as F

# 1. Definó la variable delayBinarizer
# inputCol: La columna original de retraso ('arr_delay').
# outputCol: El nombre de la nueva columna binaria ('arr_delay_binary').
# hreshold: El umbral. Valores > threshold serán 1.0, valores <= threshold serán 0.0.
delayBinarizer = Binarizer(inputCol="arr_delay", outputCol="arr_delay_binary", threshold=15.0)

# Aplicamos el Binarizer en el DataFrame flightsConvertido solo para validarlo 

df_con_binario = delayBinarizer.transform(flightsConvertido)
df_con_binario.select("arr_delay", "arr_delay_binary").show(5)


+---------+----------------+
|arr_delay|arr_delay_binary|
+---------+----------------+
|     70.0|             1.0|
|    -23.0|             0.0|
|     -4.0|             0.0|
|    -23.0|             0.0|
|     43.0|             1.0|
+---------+----------------+
only showing top 5 rows



In [66]:
assert(isinstance(delayBinarizer, Binarizer))
assert(delayBinarizer.getThreshold() == 15)
assert(delayBinarizer.getInputCol() == "arr_delay")
assert(delayBinarizer.getOutputCol() == "arr_delay_binary")

Por último, crearemos el modelo de clasificación.

* Crear en una variable `decisionTree` un árbol de clasificación de Spark (`DecisionTreeClassifier` del paquete `pyspark.ml.classification`)
* Indicar como columna de entrada la nueva columna creada por el `VectorAssembler` creado en un apartado anterior.
* Indicar como columna objetivo (target) la nueva columna creada por el `Binarizer` del apartado anterior.

In [67]:
from pyspark.ml.classification import DecisionTreeClassifier

# Por aprendizaje de otras materias en el uso de arbol de decisiones usare semillas para evitar un poco el sesgo 

decisionTree = DecisionTreeClassifier(
     featuresCol="features",
     labelCol="arr_delay_binary",
     seed=42
)

print("--- Información del Estimador DecisionTreeClassifier ---")
print("El objeto 'decisionTree' es un estimador listo para aprender.")
print("Está configurado con los siguientes parámetros:")
print(decisionTree)
print("-------------------------------------------------------")

--- Información del Estimador DecisionTreeClassifier ---
El objeto 'decisionTree' es un estimador listo para aprender.
Está configurado con los siguientes parámetros:
DecisionTreeClassifier_8609ed56dd78
-------------------------------------------------------


In [68]:
assert(isinstance(decisionTree, DecisionTreeClassifier))
assert(decisionTree.getFeaturesCol() == "features")
assert(decisionTree.getLabelCol() == "arr_delay_binary")

Ahora vamos a encapsular todas las fases en un sólo pipeline y procederemos a entrenarlo. Se pide:

* Crear en una variable llamada `pipeline` un objeto `Pipeline` de Spark con las etapas anteriores en el orden adecuado para poder entrenar un modelo. 

* Entrenarlo invocando sobre ella al método `fit` y guardar el pipeline entrenado devuelto por dicho método en una variable llamada `pipelineModel`. 

* Aplicar el pipeline entrenado para transformar (predecir) el DataFrame `flightsConvertido`, guardando las predicciones devueltas en la variable `flightsPredictions` que será un DataFrame. Nótese que estamos prediciendo los propios datos de entrenamiento y que, por simplicidad, no habíamos hecho (aunque habría sido lo correcto) ninguna división de nuestros datos originales en subconjuntos distintos de entrenamiento y test antes de entrenar.

In [69]:
# Paso 1: genero el objeto Pipeline
# Se encadeno todas las etapas en el orden lógico de procesamiento:
# StringIndexer -> Binarizer (para la etiqueta) -> VectorAssembler -> DecisionTreeClassifier
pipeline = Pipeline(stages=[
    indexerMonth,
    indexerCarrier,
    delayBinarizer,
    vectorAssembler,
    decisionTree
])

print(" :D Pipeline creado exitosamente.")
print(f"Etapas del Pipeline: {[stage.uid for stage in pipeline.getStages()]}")

# Paso 2: :D Se Entrenó el Pipeline
# Se invoco el método 'fit()' sobre el pipeline, pasándole el DataFrame de entrenamiento.
# En este caso, por simplicidad, usamos el DataFrame completo 'flightsConvertido' como entrenamiento.
pipelineModel = pipeline.fit(flightsConvertido)

print("\n :D Pipeline entrenado (ajustado a los datos) con éxito.")
print("El objeto 'pipelineModel' ahora contiene todos los transformadores ajustados y el modelo de clasificación entrenado.")


# Paso 3: :D Aplicar el Pipeline entrenado para transformar (predecir) los datos
# Se invoco el método 'transform()' sobre el pipelineModel para generar predicciones.
flightsPredictions = pipelineModel.transform(flightsConvertido)

print("\n :D  Predicciones generadas y guardadas en 'flightsPredictions' DataFrame.")
print("Este DataFrame ahora incluye las columnas de las características ensambladas,")
print("la etiqueta real ('arr_delay_binary'), la predicción ('prediction'),")
print("y las probabilidades ('probability') del modelo para cada clase.")

# Paso 4: XD Mostrar algunas de las columnas relevantes para verificar las predicciones
print("\n--- Primeras 5 filas del DataFrame de Predicciones ---")
flightsPredictions.select("arr_delay", "arr_delay_binary", "prediction", "probability", "features").show(5, truncate=False)
print("-----------------------------------------------------")

 :D Pipeline creado exitosamente.
Etapas del Pipeline: ['StringIndexer_e27bd83795fc', 'StringIndexer_e2107eca14cf', 'Binarizer_78bc66501f35', 'VectorAssembler_52fdc2a0d99c', 'DecisionTreeClassifier_8609ed56dd78']

 :D Pipeline entrenado (ajustado a los datos) con éxito.
El objeto 'pipelineModel' ahora contiene todos los transformadores ajustados y el modelo de clasificación entrenado.

 :D  Predicciones generadas y guardadas en 'flightsPredictions' DataFrame.
Este DataFrame ahora incluye las columnas de las características ensambladas,
la etiqueta real ('arr_delay_binary'), la predicción ('prediction'),
y las probabilidades ('probability') del modelo para cada clase.

--- Primeras 5 filas del DataFrame de Predicciones ---
+---------+----------------+----------+----------------------------------------+--------------------------------------+
|arr_delay|arr_delay_binary|prediction|probability                             |features                              |
+---------+----------------+

In [70]:
from pyspark.ml import PipelineModel
assert(isinstance(pipeline, Pipeline))
assert(len(pipeline.getStages()) == 5)
assert(isinstance(pipelineModel, PipelineModel))
assert("probability" in flightsPredictions.columns)
assert("prediction" in flightsPredictions.columns)
assert("rawPrediction" in flightsPredictions.columns)

Vamos a mostrar la matriz de confusión (este apartado no es evaluable). Agrupamos por la variable que tiene la clase verdadera y la que tiene la clase predicha, para ver en cuántos casos coinciden y en cuántos difieren.

In [71]:
flightsPredictions.groupBy("arr_delay_binary", "prediction").count().show()

+----------------+----------+------+
|arr_delay_binary|prediction| count|
+----------------+----------+------+
|             1.0|       1.0|   752|
|             0.0|       1.0|   181|
|             1.0|       0.0| 23497|
|             0.0|       0.0|136318|
+----------------+----------+------+

